#### 컨볼루션 신경망(CNN : Convolutional Neural Network)

In [ ]:
# CNN은 이미지뿐만 아니라 텍스트나 음성등 다양한 분야의 데이터 처리에 사용되고 있다.
# 이번 장에서는 CNN 사용법중 가장 기초가 되는 이미지 데이터를 다루는 방법을 알아보자.

# 4장에 나오는 보스턴 주택가격 데이터세트에서는 주택의 가격을 예측하기 위한 주택당 방의 개수, 재산세율, 범죄율 같은
# 특징(feature)들이 있었다. 5장에서 나온 와인 데이터세트에서도 와인의 종류나 품질을 예측하기 위한 당도, 알코올 도수
# 같은 특징들이 데이터로 존재했다.
# 이에 비해 FashionMNIST 같은 이미지 데이터에서는 어떤 특징을 찾을 수 있을까?
# 이미지 데이터에서는 연구자가 스스로 특징을 찾아야 한다. 
# 이미지 특징 추출(Feature Extraction) 기법중 하나인 컨볼루션 연산은 각 픽셀을 본래 픽셀과 그 주변 픽셀의 조합으로
# 대체하는 동작이다. 이때 연산에 사용되는 작은 행렬을 filter 또는 kernel이라고 한다.
# 컨볼루션 연산은 우리말로 합성곱이라고 한다. 원본 이미지의 각 픽셀을 포함한 주변 픽셀과 필터의 모든 픽셀을 각각
# 곱연산을 하고 그 결과들을 모두 합해서 새로운 이미지에 넣어주기 때문에 합성곱이라는 이름을 붙인 것 같다.  

In [1]:
# 간단한 컨볼루션 필터 몇 가지를 소개해보자.
# 3*3 크기의 작은 필터는 필터를 구성하는 셀마다 각각 다른 숫자를 넣어서 원본 이미지의 수직선검출, 수평선검출,
# 흐림(blur), 날카로움(sharpen) 이미지 효과등을 줄 수 있다. 
# 그런데 이런 필터들은 경험적 지식을 통해 직접 손으로 값을 넣어준 결과인 것이다. 이러한 수작업으로 설계한 특징에는
# 다음 세가지의 문제점이 있다. 첫째, 적용하고자 하는 분야에 대한 전문적 지식이 필요하다. 둘째, 수작업으로 특징을 
# 설계하는 것은 시간과 비용이 많이 드는 작업이다. 세째, 한분야(예를 들어 이미지)에서 효과적인 특징을 다른 분야
# (예를 들어 음성)에 적요하기가 어렵다.

# 딥러닝 기반의 컨볼루션 연산은 이런 문제들을 모두 해결했다. 즉 컨볼루션 신경망은 특징을 검출하는 필터를 사람이 
# 수작업으로 설계하는 것이 아니라 네트워크 스스로 특징을 추출하는 필터를 자동으로 생성한다. 
# 학습을 계속하면 네트워크를 구성하는 각 뉴런들은 입력한 데이터에 대해 특정 패턴을 잘 추출할 수 있도록 적응하게 된다.
# 그러면 컨볼루션 신경망은 어떻게 특징을 자동으로 추출하는 것일까? 컨볼루션 신경망의 주요 레이어를 하나씩 살펴보면서
# 그 방법을 알아보기로 하자.

In [ ]:
# 주요 레이어 정리

# 지금까지 배운 레이어는 Dense레이어와 Flatten레이어 두 종류였다. Dense레이어는 신경망에서 사용하는 가장 기본적인
# 레이어로 각 뉴런이 서로 완전하게 연결되어 있기 때문에 완전연결레이어(Fully Connected Layer)라고 한다. 
# Flatten레이어는 다차원의 이미지를 1차원으로 평평하게 바꿔주는 단순한 레이어이다.

# 이미지 분류에 사용되는 컨볼루션 신경망은 특징추출기(Feature Extraction)와 분류기(Classifier)가 합쳐져 있는 
# 구조이다. 특징추출기 역할을 하는 것은 convolution layer와 pooling layer이며, Dense layer는 분류기 역할이다.
# cnn -> pooling -> cnn -> pooling -> flatten -> dense ->dropout -> dense가 한가지 예이다.
# cnn과 pooling은 교차되며 배치되고, 분류기에는 Dense레이어가 배치되며, 과적합을 막기 위해 Dropout레이어가 Dense와
# Dense사이에 배치된다. 마지막 Dense레이어 뒤에는 Dropout레이어가 배치되지 않는다. 하나씩 살펴보자

In [ ]:
# 컨볼루션 레이어
# 컬볼루션 레이어(Convolution Layer)는 말 그대로 컨볼루션 연산을 하는 레이어이다. 여기서 사용되는 필터는 사람이
# 미리 정해놓은 것이 아니라 네트워크의 학습을 통해 자동으로 추출된다. 우리가 코드에서 지정해야 할 값은 필터를 구성
# 하는 각 셀들의 값들이 아니라 필터의 개수 정도이다.
# 컨볼루션 레이어는 1차원부터 3차원까지 다양하게 사용할 수 있지만 여기서는 가장 기본이 되는 2차원으로 설명한다.

# 컨볼루션 레이어는 각 채널에 대해 계산된 갑을 합쳐서 새로운 이미지를 만들어낸다. 이때 새로운 이미지의 마지막 
# 차원 수는 필터의 수와 동일하다. 일반적인 컨볼루션 신경망은 여러 개의 컨볼루션 레이어를 쌓으면서 뒤쪽으로 갈수록
# 필터의 수를 점점 늘리기 때문에 이미지의 마지막 차원 수는 점점 많아진다.